# Evolutionary Camouflage Versus a Learning Predator Population

---

**EvoCamoVsLearnPredPop.ipynb**

August 23, 2022: this version runs “local mode” with both predator and prey running on the same machine.

(The former behavior available with `Rube_Goldberg_mode = True`)

In [1]:
# "Rube Goldberg" mode refers to running camouflage evolution on my laptop while
# running predator vision in cloud via Colab. State is passed back and forth via
# files on Google Drive.

# TODO 20220822
# Rube_Goldberg_mode = True
Rube_Goldberg_mode = False

def if_RG_mode(for_RG_mode, for_normal_mode):
    return for_RG_mode if Rube_Goldberg_mode else for_normal_mode

# PredatorEye directory on Drive.
pe_directory = '/content/drive/My Drive/PredatorEye/'

# Shared "communication" directory on Drive.
shared_directory = if_RG_mode(pe_directory + 'evo_camo_vs_static_fcd/',
                              '/Users/cwr/camo_data/comms/')

# This was meant (20220716) to allow reading original pre-trained model from
# Google Drive, but I'll need to retrain it for M1 (Apple Silicon).
g_drive_pe_dir = ('/Users/cwr/Library/CloudStorage/' +
                  'GoogleDrive-craig.w.reynolds@gmail.com/' +
                  'My Drive/PredatorEye/')

# Directory for pre-trained Keras/TensorFlow models.
saved_model_directory = if_RG_mode(pe_directory, g_drive_pe_dir) + 'saved_models/'


print('Rube_Goldberg_mode =', Rube_Goldberg_mode)
print('shared_directory =', shared_directory)
print('saved_model_directory =', saved_model_directory)

# Pathname of pre-trained Keras/TensorFlow model
trained_model = saved_model_directory + '20220321_1711_FCD6_rc4'

# Directory on Drive for storing fine-tuning dataset.
fine_tuning_directory = shared_directory + 'fine_tuning/'

my_prefix = "find_"
other_prefix = "camo_"

my_suffix =  ".txt"
# other_suffix = ".jpeg"
other_suffix = ".png"

fcd_image_size = 1024
fcd_disk_size = 201

import time
import PIL
from pathlib import Path

from tensorflow import keras
import numpy as np
import random
import math

# %tensorflow_version 2.x
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

from tensorflow.keras import backend as keras_backend
keras_backend.set_image_data_format('channels_last')

# Import DiskFind utilities for PredatorEye.
import sys
if Rube_Goldberg_mode:
    sys.path.append('/content/drive/My Drive/PredatorEye/shared_code/')
else:
    sys.path.append('/Users/cwr/Documents/code/PredatorEye/')
import DiskFind as df

Rube_Goldberg_mode = False
shared_directory = /Users/cwr/camo_data/comms/
saved_model_directory = /Users/cwr/Library/CloudStorage/GoogleDrive-craig.w.reynolds@gmail.com/My Drive/PredatorEye/saved_models/
TensorFlow version: 2.9.2


# Ad hoc “predator server”

In [2]:
# Top level: wait for camo_xxx.jpeg files to appear, respond with find_xxx.txt
def start_run(step = 0):
    if step == 0:
        print('Start run in', shared_directory )
    else:
        print('Continue run at step', step, ' in', shared_directory)
    while True:
        perform_step(step, shared_directory)
        step += 1

# Continue from from the last camo_xxx.jpeg file.
def restart_run():
    start_run(newest_file_from_other(shared_directory))

# Single step: wait for camo file, write response, delete previous response.
def perform_step(step, directory):
    ############################################################################
    # TODO 20220909 add verbose "start of step" logging -- getting hard to read.
    print()
    print('step', step)
    print()
    ############################################################################
    wait_for_reply(step, shared_directory)
    write_response_file(step, shared_directory)
    delete_find_file(step - 1, shared_directory)

# Read image file for step, apply pre-trained model, write response file.
# (TODO 20220911 this has grown huge, needs to be refactored into smaller parts)
def write_response_file(step, directory):
    # Read image file and check for expected format.
    image_pathname = make_camo_pathname(step, directory)
    pixel_tensor = df.read_image_file_as_pixel_tensor(image_pathname)
    assert df.check_pixel_tensor(pixel_tensor), ('wrong file format: ' +
                                                 image_pathname)
    # TODO 20220907 can/should this TF version replace pixel_tensor below?
    #               no this causes an error in Predator.fine_tune_model()
    tf_pixel_tensor = tf.convert_to_tensor([pixel_tensor])
    
    ############################################################################
    # TODO 20220911 testing new Tournament class

#     # Choose a random tournament of three from population of Predators.
#     tournament = Predator.choose_tournament()

#     # Collect predictions from each Predator: where in image is prey located?
#     predictions_xy3 = [predator.model.predict(tf_pixel_tensor)[0]
#                        for predator in tournament]
    ############################################################################

    # Collect self-supervision signal from TexSyn: positions of prey centers.
    prey_centers_xy3 = read_3_centers_from_file(step, directory)

    ############################################################################
    # TODO 20220911 testing new Tournament class

#     # Sort predictions from the three Predators in a tournament, according to
#     # ”accuracy” (least aim error).
#     ranked_predictions_xy3 = rank_tournament(predictions_xy3, prey_centers_xy3)
    
    ttt = Tournament(pixel_tensor, prey_centers_xy3)

    # Write response file (contains 3 xy positions as 6 floats on [0,1]).
#     response_string = xy3_to_str(ranked_predictions_xy3)
    response_string = xy3_to_str(ttt.ranked_predictions())

    ############################################################################
    verify_comms_directory_reachable()
    with open(make_find_pathname(step, directory), 'w') as file:
        file.write(response_string)
    print('Wrote ' + "'" + response_string + "'",
          'to response file', Path(make_find_pathname(step, directory)).name)
    
    # Merge this step's image into fine-tuning dataset, and related bookkeeping.
    ############################################################################
    # TODO 20220911 testing new Tournament class
#     best_prediction = ranked_predictions_xy3[0]
    best_prediction = ttt.ranked_predictions()[0]
    ############################################################################
    fine_tuning_dataset.update(pixel_tensor, best_prediction, step, directory)

    # Fine-tune models of each Predator in tournament.
    ############################################################################
    # TODO 20220911 testing new Tournament class
#     for i in range(len(tournament)):
#     for i in range(ttt.size()):
#         # TODO oh this has to be inside Tournament
#         tournament[i].fine_tune_model(pixel_tensor, predictions_xy3[i], step, directory)
    
    # TODO 20220911 this really sucks
    for p in Predator.population:
        p.set_step(step)
    
    ttt.fine_tune_models()
    ############################################################################


# Delete the given file, presumably after having written the next one.
def delete_find_file(step, directory):
    # Why doesn't pathlib provide a Path.remove() method like os?
    # TODO oh, missing_ok was added at pathlib version 3.8.
    # Path(makeMyPathname(step, directory)).unlink(missing_ok=True)
    p = Path(make_find_pathname(step, directory))
    if p.exists():
        p.unlink()

# Delete any remaining file in commuications directory to start a new run.
def clean_up_communication_directory():
    def delete_directory_contents(directory_path):
        for path in directory_path.iterdir():
            print('Removing from communication directory:', path)
            if path.is_dir():
                delete_directory_contents(path)
                path.rmdir()
            else:
                path.unlink()
    delete_directory_contents(Path(shared_directory))

# From pathname for file of given step number from the "other" agent.
def make_camo_pathname(step, directory):
    return directory + other_prefix + str(step) + other_suffix

# Form pathname for "find_xx.txt" response file from "this" agent.
def make_find_pathname(step, directory):
    return directory + my_prefix + str(step) + my_suffix

# Form pathname for "prey_xx.txt" ground truth file from "other" agent.
def make_prey_pathname(step, directory):
    return directory + 'prey_' + str(step) + '.txt'

# Used to ping the comms directory when it seems hung.
def write_ping_file(count, step, directory):
    pn = directory + 'ping_cloud_' + str(step) + '.txt'
    verify_comms_directory_reachable()
    with open(pn, 'w') as file:
        file.write(str(count))
    print('Ping comms: ', count, pn)

# Wait until other agent's file for given step appears.
def wait_for_reply(step, directory):
    camo_pathname = Path(make_camo_pathname(step, directory))
    camo_filename = camo_pathname.name
    prey_pathname = Path(make_prey_pathname(step, directory))
    prey_filename = prey_pathname.name
    print('Waiting for', camo_filename, 'and', prey_filename, '...',
          end='', flush=True)
    start_time = time.time()
    # Loop until both files are present, waiting 1 second between tests.
    test_count = 0
    while not (is_file_present(camo_pathname) and
               is_file_present(prey_pathname)):
        time.sleep(1)
        test_count += 1
        if (test_count % 100) == 0:
            write_ping_file(test_count, step, directory)
    print(' done, elapsed time:', int(time.time() - start_time), 'seconds.')

# Like fs::exists()
def is_file_present(file):
    result = False
    verify_comms_directory_reachable()
    filename = Path(file).name
    directory = Path(file).parent
    for i in directory.iterdir():
        if i.name == filename:
            result = True
    return result

# Returns the step number of the newest file from "other" in given directory.
# (So if "camo_573.jpeg" is the only "other" file there, returns int 573)
def newest_file_from_other(directory):
    steps = [0]  # Default to zero in case dir is empty.
    for filename in Path(directory).iterdir():
        name = filename.name
        if other_prefix == name[0:len(other_prefix)]:
            steps.append(int(name.split(".")[0].split("_")[1]))
    return max(steps)

# Read ground truth prey center location data provided in "prey_n.txt" file.
def read_3_centers_from_file(step, directory):
    # Read contents of file as string.
    verify_comms_directory_reachable()
    with open(make_prey_pathname(step, directory), 'r') as file:
        prey_centers_string = file.read()
    # Split string at whitespace, map to 6 floats, reshape into 3 xy pairs.
    # (TODO could probably be rewritten cleaner with "list comprehension")
    array = np.reshape(list(map(float, prey_centers_string.split())), (3, 2))
    return array.tolist()

# Keep log of in_disk metric.
def write_in_disk_log(step, history):
    if step % 10 == 0:
        in_disk = history.history["in_disk"][0]
        pathname = shared_directory + 'in_disk_log.csv'
        verify_comms_directory_reachable()
        with open(pathname, 'a') as file:
            if step == 0:
                file.write('step,in_disk\n')
            file.write(str(step) + ',' + "{:.4f}".format(in_disk) + '\n')

# Just wait in retry loop if shared "comms" directory become unreachable.
# Probably will return shortly, better to wait than signal a file error.
# (This is called from places with a local "directory" but it uses global value.)
def verify_comms_directory_reachable():
    seconds = 0
    # shared_directory_pathname = Path(shared_directory)
    # while not shared_directory_pathname.is_dir():
    while not Path(shared_directory).is_dir():
        print("Shared “comms” directory,", shared_directory, 
              "has been inaccessible for", seconds, "seconds.")
        time.sleep(1)  # wait 1 sec
        seconds += 1

# Given 3 prey positions ("xy3"), sort them by proximity to "point" (prediction)
def sort_xy3_by_proximity_to_point(xy3, point):
    # print('xy3 =', xy3)
    xy3_plus_distance = [[df.dist2d(xy, point), xy] for xy in xy3]
    # print('xy3_plus_distance =', xy3_plus_distance)
    sorted_xy3_plus_key = sorted(xy3_plus_distance, key=lambda x: x[0])
    # print('sorted_xy3_plus_key =', sorted_xy3_plus_key)
    sorted_xy3 = [x[1] for x in sorted_xy3_plus_key]
    # print('sorted_xy3 =', sorted_xy3)
    return sorted_xy3

# Convert xy3 to string: [[x,y], [p,q], [r,s]] -> 'x y p q r s '
def xy3_to_str(xy3):
    return ''.join('%s ' % i for i in flatten_nested_list(xy3))

def flatten_nested_list(nested_list):
    return [item for sublist in nested_list for item in sublist]

# Read pre-trained model

As I integrate this into the Predator class, this is no longer “Read pre-trained model” but more like “Some utilities for reading the pre-trained model”

In [3]:
# Read pre-trained TensorFlow "predator vision" model.

# print('Reading pre-trained model from:', trained_model)

# ad hoc workaround suggested on https://stackoverflow.com/q/66408995/1991373
#
# dependencies = {
#     'hamming_loss': tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"),
#     'attention': attention(return_sequences=True)
# }
#
# dependencies = {
#     'valid_accuracy': ValidAccuracy
# }

# Calculates RELATIVE disk radius on the fly -- rewrite later.
def fcd_disk_radius():
    return (float(fcd_disk_size) / float(fcd_image_size)) / 2

# Given two tensors of 2d point coordinates, return a tensor of the Cartesian
# distance between corresponding points in the input tensors.
def corresponding_distances(y_true, y_pred):
    true_pos_x, true_pos_y = tf.split(y_true, num_or_size_splits=2, axis=1)
    pred_pos_x, pred_pos_y = tf.split(y_pred, num_or_size_splits=2, axis=1)
    dx = true_pos_x - pred_pos_x
    dy = true_pos_y - pred_pos_y
    distances = tf.sqrt(tf.square(dx) + tf.square(dy))
    return distances

# 20211231 copied from Find_Concpocuous_Disk
def in_disk(y_true, y_pred):
    distances = corresponding_distances(y_true, y_pred)
    # relative_disk_radius = (float(fcd_disk_size) / float(fcd_image_size)) / 2

    # From https://stackoverflow.com/a/42450565/1991373
    # Boolean tensor marking where distances are less than relative_disk_radius.
    # insides = tf.less(distances, relative_disk_radius)
    insides = tf.less(distances, fcd_disk_radius())
    map_to_zero_or_one = tf.cast(insides, tf.int32)
    return map_to_zero_or_one

dependencies = { 'in_disk': in_disk }

def read_default_pre_trained_model():
    print('Reading pre-trained model from:', trained_model)
    return keras.models.load_model(trained_model, custom_objects=dependencies)

# FineTuningDataset

In [4]:
class FineTuningDataset:
    """Manages the dataset of images and labels for fine-tuning."""

    # Accumulated a new “training set” of the most recent N steps seen so far. (See
    # https://cwreynolds.github.io/TexSyn/#20220421 and ...#20220424 for discussion
    # of this parameter. Had been 1, then 100, then 200, then finally, infinity.) 
    # max_training_set_size = float('inf') # keep ALL steps in training set, use GPU.
    max_training_set_size = 500 # Try smaller again, "yellow flowers" keeps failing.
    # List of "pixel tensors".
    fine_tune_images = []
    # List of xy3 [[x,y],[x,y],[x,y]] for 3 prey centers.
    fine_tune_labels = []

    def update(self, pixel_tensor, prediction, step, directory):
        # Assume the predator was "aiming for" that one but missed by a bit.
        xy3 = read_3_centers_from_file(step, directory)
        sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)

        # Accumulate the most recent "max_training_set_size" training samples.
        self.fine_tune_images.append(pixel_tensor)
        self.fine_tune_labels.append(sorted_xy3)

        # If training set has become too large, slice off first element of each.
        if len(self.fine_tune_images) > self.max_training_set_size:
            self.fine_tune_images = self.fine_tune_images[1:]
            self.fine_tune_labels = self.fine_tune_labels[1:]

        print('  fine_tune_images shape =', np.shape(self.fine_tune_images),
              '-- fine_tune_labels shape =', np.shape(self.fine_tune_labels))
        

# Create a global FineTuningDataset object.
# (TODO globals are usually a bad idea, reconsider this.)
fine_tuning_dataset = FineTuningDataset()

# Predator class

In [5]:
class Predator:
    """Represents a Predator in the camouflage simulation. It has a CNN-based
       model of visual hunting that identified the position of likely prey."""

    # Global list of active Predators. (As a class variable.)
    population = []
    
    ############################################################################
    # TODO 20220905 cache default_pre_trained_model
    default_pre_trained_model = read_default_pre_trained_model()
    ############################################################################


    def __init__(self):
        # Each Predator has its own copy of a prey-finding CNN-based model.
        self.model = []
        # By default add this new Predator to the population (TODO reconsider)
        Predator.population.append(self)
        ########################################################################
        # TODO 20220909
        # keep history of predation events: was hunt a success or a failure?
        self.successes = []
        ########################################################################



    ############################################################################
    # TODO 20220905 cache default_pre_trained_model
    
#     # Set this Predator's model to default.
#     def initialize_to_pre_trained_model(self):
#         self.model = read_default_pre_trained_model()

#     # Set this Predator's model to default.
#     def initialize_to_pre_trained_model(self):
#         self.model = self.default_pre_trained_model

#     # TODO 20220905 changed back because was just copying pointer not deep copy.
#     def initialize_to_pre_trained_model(self):
#         self.model = read_default_pre_trained_model()
    ############################################################################

    # Keep track of how often selected prey is nearest center:
    nearest_center = 0
    
    # TODO 20220911 this is a goofy hack to avoid passing the "step" parameter
    # to Predator.fine_tune_model() for the sole purpose of logging about
    # "nearest_center" which really needs to be redesigned.
    def set_step(self, step):
        self.step = step

    # Apply fine-tuning to (originally pre-trained) predator. Use recent steps as
    # training set. Assume they were "near misses" and so training label is actual
    # (ground truth) center of disk nearest prediction. Keep a max number of old
    # steps to allow gradually forgetting the earliest part of the run.
    
#123456789 123456789 123456789 123456789 123456789 123456789 123456789 123456789

    ############################################################################
    # TODO 20220911 remove dependency of Predator.fine_tune_model() on step and
    #               directory args. replace them with prey_centers_xy3
#     def fine_tune_model(self, pixel_tensor, prediction, step, directory):
    def fine_tune_model(self, pixel_tensor, prediction, prey_centers_xy3):

        # Assume the predator was "aiming for" that one but missed by a bit.
#         xy3 = read_3_centers_from_file(step, directory)
#         sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)
        sorted_xy3 = sort_xy3_by_proximity_to_point(prey_centers_xy3, prediction)


        # keep track of how often selected prey is nearest center:
#         temp = xy3.copy()  # needed? (much later 20220911, no I don't think so)
        temp = prey_centers_xy3.copy()  # needed? (much later 20220911, no I don't think so)

        sorted_by_dist_to_center = sort_xy3_by_proximity_to_point(temp, [0.5, 0.5])
        if sorted_by_dist_to_center[0] == sorted_xy3[0]:
            Predator.nearest_center += 1
        
        # TODO 20220829 "ad-hoc-ly" adjust for two models running, hence NC count doubled.
        # print('  nearest_center:',
        #       str(int(100 * float(self.nearest_center) / (step + 1))) + '%',
        #       '(nearest_center =', self.nearest_center, ', steps =', step + 1, ')')
        nc = self.nearest_center / 2
#         print('  nearest_center:',
#               str(int(100 * float(nc) / (step + 1))) + '%',
#               '(nearest_center =', nc, ', steps =', step + 1, ')')
        print('  nearest_center:',
              str(int(100 * float(nc) / (self.step + 1))) + '%',
              '(nearest_center =', nc, ', steps =', self.step + 1, ')')

        ########################################################################


        # Convert training data list to np arrays
        images_array = np.array(fine_tuning_dataset.fine_tune_images)
        labels_array = np.array([x[0] for x in fine_tuning_dataset.fine_tune_labels])

        # print('images_array.shape =', images_array.shape,
        #       '-- labels_array.shape =', labels_array.shape)

    	# Skip fine-tuning until dataset is large enough (10% of max size).
        ########################################################################
        # TODO 20220803 clone model

        # print('disabled "Skip fine-tuning until dataset is large enough"')
        # # if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):
        # if images_array.shape[0] > 0: ##########################################
        #     # Do fine-tuning training step using data accumulated during run.
        #     history = self.model.fit(x=images_array, y=labels_array)
        #     # Keep log of in_disk metric:
        #     write_in_disk_log(step, history)

        # print('disabled "Skip fine-tuning until dataset is large enough"')
        # # if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):
        # if images_array.shape[0] > 0: ##########################################
        #     # Do fine-tuning training step using data accumulated during run.
        #     history = self.model.fit(x=images_array, y=labels_array)
        #     # Keep log of in_disk metric:
        #     write_in_disk_log(step, history)

        # TODO 20220823 restore "Skip fine-tuning until dataset is large enough"
        
#         print('disabled "Skip fine-tuning until dataset is large enough"')
#         if images_array.shape[0] > 0: ##########################################

        if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):

            # # Do fine-tuning training step using data accumulated during run.
            # history = self.model.fit(x=images_array, y=labels_array)
            
            # TODO 20220823 -- run fine-tuning on CPU only.
            print('Running on CPU ONLY!')
            with tf.device('/cpu:0'):
                # Do fine-tuning training step using data accumulated during run.
                history = self.model.fit(x=images_array, y=labels_array)
            
            # Keep log of in_disk metric:
#             write_in_disk_log(step, history)
            write_in_disk_log(self.step, history)


        print(end='  ')
        self.print_model_trace()
        ########################################################################
        
        
        # TODO 20220911 can this be moved to end of fine_tune_model()?
#         for i in range(3):
#              tournament[i].record_predation_success(predictions_xy3[i], prey_centers_xy3[i])

#         # TODO 20220911 moved this here from write_response_file(), are these the correct args?
#         self.record_predation_success(prediction, xy3)

        self.record_predation_success(prediction, prey_centers_xy3)


    ############################################################################



    
    ############################################################################
    # TODO 20220803 clone model

    # # Copy the neural net model of a given predator into this one.
    # def copy_model(self, another_predator):
    #     # No this is wrong, just does a shallow copy
    #     # self.model = another_predator.model
    #     self.model = tf.keras.models.clone_model(another_predator.model)
    #     # Compile newly cloned model.
    #     # compile_disk_finder_model(self.model)
    #     df.compile_disk_finder_model(self.model)

#     # Copy the neural net model of a given predator into this one. (From "Make
#     # deep copy of keras model" https://stackoverflow.com/a/54368176/1991373)
#     def copy_model(self, another_predator):
#         other_model = another_predator.model
#         # Clone layer structure of other model.
#         # self.model = tf.keras.models.clone_model(another_predator.model)
#         self.model = tf.keras.models.clone_model(other_model)
#         # Compile newly cloned model.
#         df.compile_disk_finder_model(self.model)

#         # Copy weights of other model.
#         self.model.set_weights(other_model.get_weights())


    # TODO 20220907 refactor into two methods

    # Copy the neural net model of a given predator into this one. (From "Make
    # deep copy of keras model" https://stackoverflow.com/a/54368176/1991373)
    
    def copy_model_of_another_predator(self, another_predator):
        copy_model(self, another_predator.model)

    
#     def copy_model(self, another_predator):
#         other_model = another_predator.model
    def copy_model(self, other_model):
        # Clone layer structure of other model.
        self.model = tf.keras.models.clone_model(other_model)
        # Compile newly cloned model.
        df.compile_disk_finder_model(self.model)
        # Copy weights of other model.
        self.model.set_weights(other_model.get_weights())

    ############################################################################

    # Modify this Predator's model by adding noise to its weights.
#     def jiggle_model(self):
#         weight_perturbation(self.model, 0.001)
############################################################################
# TODO 20220909 trying to control TF retracing
#     def jiggle_model(self, strength = 0.001):
#         weight_perturbation(self.model, strength)
    def jiggle_model(self, strength = 0.001):
        weight_perturbation(self.model, tf.constant(strength))
############################################################################

    
    # Print the "first" weight of each layer of this Predator's Keras model.
    def print_model_trace(self):
        for layer in self.model.layers:
            trainable_weights = layer.trainable_variables
            for weight in trainable_weights:            
                weight_shape = tf.shape(weight)
                total_size = tf.math.reduce_prod(weight_shape)
                reshape_1d = tf.reshape(weight, [total_size])
                
                # TODO 20220829 take "middle" not first parameter.
                # value = reshape_1d[0].numpy()
                middle = math.floor(total_size / 2)
                value = reshape_1d[middle].numpy()
        
                # print(round(value, 2), end = " ")
                print(round(value, 3), end = " ")

        print()

    ############################################################################
    # TODO 20220905
    # Create the given number of Predator objects
    # (TODO maybe the pretrained model should be an arg?)
    def initialize_predator_population(population_size):
        for i in range(population_size):
            p = Predator()
            print('Predator instance address:', id(p))

#             p.initialize_to_pre_trained_model()
            # TODO 20220907 maybe it sould just do this by default in constructor?
            p.copy_model(Predator.default_pre_trained_model)
        
            p.print_model_trace()


            
#             p.jiggle_model()
#             p.jiggle_model(0.1)
#             p.jiggle_model(0.01)
#             p.jiggle_model(0.001)
            p.jiggle_model(0.003)




            p.print_model_trace()


            # does this help?
            # Compile newly cloned model.
            df.compile_disk_finder_model(p.model)

            p.print_model_trace()
        print('Created population of', population_size, 'Predators.')
    ############################################################################
    
    ######################################################################################
#     # TODO 20220907 prototype tournament maker, eventually random, now just first 3
#     def choose_tournament(size = 3):
#         tournament = []
#         for i in range(size):
#             tournament.append(Predator.population[i])
#         return tournament

    # TODO 20220909 do real random selection of tournament from population

    # TODO 20220907 prototype tournament maker, eventually random, now just first 3
    def choose_tournament(size = 3):
#         tournament = []
#         for i in range(size):
#             tournament.append(Predator.population[i])
        assert len(Predator.population) >= size, "Population smaller than tournament size."
        return random.sample(Predator.population, size)


    ######################################################################################

    
    ######################################################################################
    # TODO 20220909 keep history of predation events: was hunt a success or a failure?
    
#     # Given one predator's xy prediction, and three prey center positions, find the
#     # "aim error": the distance from the predition to the nearest prey center.
#     def aim_error(prediction_xy, prey_centers_xy3):

    
#     def record_predation_success(self, prediction_xy, prey_centers_xy3):
# #         success = 0
#         ae = aim_error(prediction_xy, prey_centers_xy3)
# #         if ae < prey_radius:
# #             success = 1
#         variable = 0 if ae < prey_radius else 1
#         successes.append(success)

    def record_predation_success(self, prediction_xy, prey_centers_xy3):
        radius = df.relative_disk_radius()
        distance = aim_error(prediction_xy, prey_centers_xy3)
        self.successes.append(0 if distance < radius else 1)
        self.successes = self.successes[-10:] # Trim to just last 10 of history
#         print('successes =', self.successes)
#         starve = (self.successes[-1] + self.successes[-2]) == 0
        starve = ((len(self.successes) >= 2) and
                  ((self.successes[-1] + self.successes[-2]) == 0))
        print('successes =', self.successes, "starve!" if starve else "")



    ######################################################################################





# Utility based on https://stackoverflow.com/a/64542651/1991373
# TODO 20220907 added this to avoid always getting the same random_weights
# @tf.function

@tf.function(reduce_retracing=True)
def weight_perturbation(model, max_range):
    """Add noise to all weights in a Keras model."""
    for layer in model.layers:
        trainable_weights = layer.trainable_variables
        for weight in trainable_weights:
            random_weights = tf.random.uniform(tf.shape(weight),
                                               max_range / -2,
                                               max_range / 2,
                                               dtype=tf.float32)
            weight.assign_add(random_weights)

            
# print(tf.random.uniform((2, 2), 1 / -2, 1 / 2, dtype=tf.float32))
# print(tf.random.uniform((2, 2), 1 / -2, 1 / 2, dtype=tf.float32))
# print()

# TODO 20220905 -- this should probably not be at global scope:
# Create population of 12 Predators.
Predator.initialize_predator_population(12)
# Predator.initialize_predator_population(3)
print('population size:', len(Predator.population))

Reading pre-trained model from: /Users/cwr/Library/CloudStorage/GoogleDrive-craig.w.reynolds@gmail.com/My Drive/PredatorEye/saved_models/20220321_1711_FCD6_rc4
Predator instance address: 5952658160
-0.391 -0.008 -0.108 -0.969 -0.191 -0.095 0.104 0.01 -0.005 -3.86 0.29 1.045 -0.161 -5.435 -0.02 6.254 0.028 0.211 
-0.392 -0.009 -0.108 -0.97 -0.191 -0.094 0.105 0.008 -0.005 -3.859 0.291 1.045 -0.161 -5.436 -0.02 6.255 0.029 0.209 
-0.392 -0.009 -0.108 -0.97 -0.191 -0.094 0.105 0.008 -0.005 -3.859 0.291 1.045 -0.161 -5.436 -0.02 6.255 0.029 0.209 
Predator instance address: 6074142576
-0.391 -0.008 -0.108 -0.969 -0.191 -0.095 0.104 0.01 -0.005 -3.86 0.29 1.045 -0.161 -5.435 -0.02 6.254 0.028 0.211 
-0.391 -0.008 -0.108 -0.969 -0.191 -0.096 0.104 0.011 -0.007 -3.859 0.289 1.046 -0.161 -5.436 -0.021 6.254 0.03 0.209 
-0.391 -0.008 -0.108 -0.969 -0.191 -0.096 0.104 0.011 -0.007 -3.859 0.289 1.046 -0.161 -5.436 -0.021 6.254 0.03 0.209 
Predator instance address: 6051353984
-0.391 -0.008 -0.108

In [6]:
# foo = [1, 2, 3, 4, 5, 6]
# foo = [1, 2, 3]

# bar = foo[5:]
# baz = foo[-5:]

# print('foo', foo)
# print('bar', bar)
# print('baz', baz)

random.sample([2,4,6,8,10], 3)

[10, 6, 2]

# Prototype rank tournament of Predators by min prediction-prey distance 


In [7]:
# Order a tournamant of 3 Predators by sorting their three xy predictions. They
# are ordered by distance from prediction to the nearest of three prey centers
# (smaller being earlier in sort order -- think of it as "aiming error").

def rank_tournament(predictions_xy3, prey_centers_xy3):
    return sort_xy3_by_least_aim_error(predictions_xy3, prey_centers_xy3)

# Given 3 predator prediction positions ("xy3"), sort them by "least aim error",
# that is, the distance from the prediction to the nearest prey position, so
# smallest errors sort to the front of the collection.

def sort_xy3_by_least_aim_error(predictions_xy3, prey_centers_xy3):
    predictions_with_key = [[aim_error(xy, prey_centers_xy3), xy]
                            for xy in predictions_xy3]
    # print('predictions_with_key =', predictions_with_key)
    sorted_predictions_with_key = sorted(predictions_with_key,
                                         key=lambda x: x[0])
    # print('sorted_predictions_with_key =', sorted_predictions_with_key)
    sorted_predictions = [x[1] for x in sorted_predictions_with_key]
    # print('sorted_predictions =', sorted_predictions)
    return sorted_predictions

# Given one predator's xy prediction, and three prey center positions, find the
# "aim error": the distance from the predition to the nearest prey center.
def aim_error(prediction_xy, prey_centers_xy3):
    min_aim_error = math.inf
    for xy in prey_centers_xy3:
        distance = df.dist2d(xy, prediction_xy)
        if min_aim_error > distance:
            min_aim_error = distance
    return min_aim_error


# rank_tournament([[0.2, 0.5], [0.4, 0.5], [0.3, 0.5]],
#                 [[0.5, 0.5], [0.6, 0.5], [0.7, 0.5]])

In [8]:
# TODO 20220911 working to make a Tournament class to encapsulate most of what
#               is currently in write_response_file()

# def write_response_file(step, directory):
#     # Read image file and check for expected format.
#     image_pathname = make_camo_pathname(step, directory)
#     pixel_tensor = df.read_image_file_as_pixel_tensor(image_pathname)
#     assert df.check_pixel_tensor(pixel_tensor), ('wrong file format: ' +
#                                                  image_pathname)
#     # TODO 20220907 can/should this TF version replace pixel_tensor below?
#     #               no this causes an error in Predator.fine_tune_model()
#     tf_pixel_tensor = tf.convert_to_tensor([pixel_tensor])
    
#     # Choose a random tournament of three from population of Predators.
#     tournament = Predator.choose_tournament()

#     # Collect predictions from each Predator: where in image is prey located?
#     predictions_xy3 = [predator.model.predict(tf_pixel_tensor)[0]
#                        for predator in tournament]
    
#     # Collect self-supervision signal from TexSyn: positions of prey centers.
#     prey_centers_xy3 = read_3_centers_from_file(step, directory)

#     # Sort predictions from the three Predators in a tournament, according to
#     # ”accuracy” (least aim error).
#     ranked_predictions_xy3 = rank_tournament(predictions_xy3, prey_centers_xy3)

#     # Write response file (contains 3 xy positions as 6 floats on [0,1]).
#     response_string = xy3_to_str(ranked_predictions_xy3)
#     verify_comms_directory_reachable()
#     with open(make_find_pathname(step, directory), 'w') as file:
#         file.write(response_string)
#     print('Wrote ' + "'" + response_string + "'",
#           'to response file', Path(make_find_pathname(step, directory)).name)
    
#     # Merge this step's image into fine-tuning dataset, and related bookkeeping.
#     best_prediction = ranked_predictions_xy3[0]
#     fine_tuning_dataset.update(pixel_tensor, best_prediction, step, directory)

#     # Fine-tune models of each Predator in tournament.
#     for i in range(len(tournament)):
#         tournament[i].fine_tune_model(pixel_tensor, predictions_xy3[i], step, directory)

#123456789 123456789 123456789 123456789 123456789 123456789 123456789 123456789

#     class Tournament:
#         """Represents three Predators in a Tournament of the camouflage simulation."""

#         def __init__(self, pixel_tensor, prey_centers_xy3):
#             # Store pixel data from current input image from TexSyn side.
#             self.pixel_tensor = pixel_tensor

#             # Also store it as a TF-style tensor        
#             # TODO 20220907 should eventually replace the non-TF data.
#             #               now causes an error in Predator.fine_tune_model()
#             self.tf_pixel_tensor = tf.convert_to_tensor([self.pixel_tensor])

#     #         # Collect self-supervision signal from TexSyn: positions of prey centers.
#     #         prey_centers_xy3 = read_3_centers_from_file(step, directory)
#             # Store the positions of all prey centers as xy on image.
#             self.prey_centers_xy3 = prey_centers_xy3

#             # Choose a random tournament of three from population of Predators.
#             self.tournament = Predator.choose_tournament()

#             # Collect predictions from each Predator: where in image is prey located?
#             self.predictions_xy3 = [predator.model.predict(self.tf_pixel_tensor)[0]
#                                     for predator in self.tournament]

#             # Sort predictions from the three Predators in a tournament, according to
#             # ”accuracy” (least aim error).


#             ########################################################################
#             # TODO 20220912 are we changing the order of these?
#             print('predictions_xy3', self.predictions_xy3)
#             ########################################################################

#             self.ranked_predictions_xy3 = rank_tournament(self.predictions_xy3,
#                                                           prey_centers_xy3)

#             ########################################################################
#             # TODO 20220912 are we changing the order of these?
#             print('predictions_xy3', self.predictions_xy3)
#             ########################################################################


#     #     # How many Predators in this tournament. Will "always" be 3.
#     #     def size(self):
#     #         return len(self.tournament)

#         # Gets the list of 3 prey center predictions, ranked most accurate at front.
#         def ranked_predictions(self):
#             return self.ranked_predictions_xy3

#     #     # Perform fine-tuning step on each Predator in this Tournament.
#     #     def fine_tune_models(self):        
#     #         for i in range(len(self.tournament)):
#     # #             tournament[i].fine_tune_model(pixel_tensor, predictions_xy3[i], step, directory)
#     #             self.tournament[i].fine_tune_model(self.pixel_tensor,
#     #                                                self.predictions_xy3[i],
#     #                                                self.prey_centers_xy3)

#         # Perform fine-tuning step on each Predator in this Tournament.
#         def fine_tune_models(self):        
#             for i in range(len(self.tournament)):
#                 self.tournament[i].fine_tune_model(self.pixel_tensor,
#                                                    self.predictions_xy3[i],
#                                                    self.prey_centers_xy3)



#123456789 123456789 123456789 123456789 123456789 123456789 123456789 123456789


# TODO 20220913 make class Member inside class Tournament.


class Tournament:
    """Represents three Predators in a Tournament of the camouflage simulation."""
    
    class Member:
        """One Predator in a Tournament of the camouflage simulation."""
        def __init__(self, predator, tf_pixel_tensor, prey_centers_xy3):
            self.predator = predator
            self.tf_pixel_tensor = tf_pixel_tensor
            self.prey_centers_xy3 = prey_centers_xy3
            
            self.prediction = self.predator.model.predict(tf_pixel_tensor)[0]
            
            ################ needs to compute and store aim_error ######################################
            
            #         def :


            self.aim_error = aim_error(self.prediction, self.prey_centers_xy3)



    def __init__(self, pixel_tensor, prey_centers_xy3):
        # Store pixel data from current input image from TexSyn side.
        self.pixel_tensor = pixel_tensor
        
        # Also store it as a TF-style tensor        
        # TODO 20220907 should eventually replace the non-TF data.
        #               now causes an error in Predator.fine_tune_model()
        self.tf_pixel_tensor = tf.convert_to_tensor([self.pixel_tensor])
        
        # Store the positions of all prey centers as xy on image.
        self.prey_centers_xy3 = prey_centers_xy3
        
#         # Choose a random tournament of three from population of Predators.
#         #     TODO should this be called "members"?
#         #     TODO pass explicit size of 3?
#         self.tournament = Predator.choose_tournament()

#         # Choose a random tournament of three from population of Predators.
#         predators = Predator.choose_tournament(3)
#         self.members = [Member(predator, pixel_tensor, prey_centers_xy3)
#                         for predator in predators]

        # Choose random tournament of three from population of Predators.
        # Build a list with each wrapped in Tournament Member objects.
        self.members = [self.Member(predator, self.tf_pixel_tensor, prey_centers_xy3)
                        for predator in Predator.choose_tournament(3)]

#         # Collect predictions from each Predator: where in image is prey located?
#         self.predictions_xy3 = [predator.model.predict(self.tf_pixel_tensor)[0]
#                                 for predator in self.tournament]

        # Collect predictions from each Predator: where in image is prey located?
        # TODO is this still needed?
        self.predictions_xy3 = [member.prediction for member in self.members]

        
        
#         # from above, for temp reference
#         def sort_xy3_by_least_aim_error(predictions_xy3, prey_centers_xy3):
#             predictions_with_key = [[aim_error(xy, prey_centers_xy3), xy]
#                                     for xy in predictions_xy3]
#             # print('predictions_with_key =', predictions_with_key)
#             sorted_predictions_with_key = sorted(predictions_with_key,
#                                                  key=lambda x: x[0])
#             # print('sorted_predictions_with_key =', sorted_predictions_with_key)
#             sorted_predictions = [x[1] for x in sorted_predictions_with_key]
#             # print('sorted_predictions =', sorted_predictions)
#             return sorted_predictions
#
#         def aim_error(prediction_xy, prey_centers_xy3):

        # Rank members by aim_error (smallest first)
        # TODO should it just overwrite self.members or have another self.ranked_members?
        self.members = sorted(self.members, key=lambda m: m.aim_error)

        ############ !!!!!!!!!!!!!!! in process of converting to Members !!!!!!!!!!!!!!!!!!!####################


#         # Sort predictions from the three Predators in a tournament, according to
#         # ”accuracy” (least aim error).
#         self.ranked_predictions_xy3 = rank_tournament(self.predictions_xy3,
#                                                       prey_centers_xy3)

        # Sort predictions from the three Predators in a tournament, according to
        # ”accuracy” (least aim error).
        self.ranked_predictions_xy3 = [member.prediction for member in self.members]



    # Gets the list of 3 prey center predictions, ranked most accurate at front.
    def ranked_predictions(self):
        return self.ranked_predictions_xy3
    
#     # Perform fine-tuning step on each Predator in this Tournament.
#     def fine_tune_models(self):        
#         for i in range(len(self.tournament)):
#             self.tournament[i].fine_tune_model(self.pixel_tensor,
#                                                self.predictions_xy3[i],
#                                                self.prey_centers_xy3)

    # Perform fine-tuning step on each Predator in this Tournament.
    def fine_tune_models(self):        
#         for i in range(len(self.tournament)):
        for i in range(len(self.members)):
#             self.tournament[i].fine_tune_model(self.pixel_tensor,
            self.members[i].predator.fine_tune_model(self.pixel_tensor,
                                                     self.predictions_xy3[i],
                                                     self.prey_centers_xy3)




#123456789 123456789 123456789 123456789 123456789 123456789 123456789 123456789





# Run test

In [9]:
# # TODO 20220827 testing print_model_trace
# test_predator.print_model_trace()



# Keep track of how often selected prey is nearest center:
Predator.nearest_center = 0

# Predator.population = []
# TODO maybe a Predator.reset() method?

# Flush out obsolete files in comms directory.
clean_up_communication_directory()

# Start fresh run defaulting to step 0.
start_run()

Removing from communication directory: /Users/cwr/camo_data/comms/find_0.txt
Removing from communication directory: /Users/cwr/camo_data/comms/ping_earth_1.txt
Removing from communication directory: /Users/cwr/camo_data/comms/prey_1.txt
Removing from communication directory: /Users/cwr/camo_data/comms/prey_0.txt
Removing from communication directory: /Users/cwr/camo_data/comms/camo_1.png
Start run in /Users/cwr/camo_data/comms/

step 0

Waiting for camo_0.png and prey_0.txt ... done, elapsed time: 5 seconds.
1/1 [==============================] - 0s 79ms/step
Wrote '0.53252333 0.58855736 0.5348571 0.5764504 0.52609295 0.5669747 ' to response file find_0.txt
  fine_tune_images shape = (1, 128, 128, 3) -- fine_tune_labels shape = (1, 3, 2)
  nearest_center: 50% (nearest_center = 0.5 , steps = 1 )
  -0.391 -0.007 -0.108 -0.97 -0.191 -0.096 0.105 0.009 -0.005 -3.859 0.29 1.045 -0.16 -5.435 -0.02 6.252 0.03 0.212 
successes = [1] 
  nearest_center: 100% (nearest_center = 1.0 , steps = 1 )
 

  -0.392 -0.008 -0.106 -0.968 -0.19 -0.097 0.105 0.01 -0.004 -3.86 0.289 1.046 -0.162 -5.436 -0.021 6.254 0.027 0.21 
successes = [1, 1] 

step 7

Waiting for camo_7.png and prey_7.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 99ms/step
Wrote '0.26853397 0.46054316 0.25383234 0.4650697 0.25879228 0.47003737 ' to response file find_7.txt
  fine_tune_images shape = (8, 128, 128, 3) -- fine_tune_labels shape = (8, 3, 2)
  nearest_center: 56% (nearest_center = 4.5 , steps = 8 )
  -0.391 -0.008 -0.108 -0.969 -0.191 -0.096 0.104 0.011 -0.007 -3.859 0.289 1.046 -0.161 -5.436 -0.021 6.254 0.03 0.209 
successes = [1] 
  nearest_center: 62% (nearest_center = 5.0 , steps = 8 )
  -0.391 -0.007 -0.108 -0.97 -0.191 -0.096 0.105 0.009 -0.005 -3.859 0.29 1.045 -0.16 -5.435 -0.02 6.252 0.03 0.212 
successes = [1, 1, 1] 
  nearest_center: 68% (nearest_center = 5.5 , steps = 8 )
  -0.39 -0.008 -0.107 -0.969 -0.19 -0.095 0.105 0.009 -0.006 -3.861 0.29 1.046 -0.161 -5.435

  -0.391 -0.007 -0.108 -0.97 -0.191 -0.096 0.105 0.009 -0.005 -3.859 0.29 1.045 -0.16 -5.435 -0.02 6.252 0.03 0.212 
successes = [1, 1, 1, 0, 1] 

step 15

Waiting for camo_15.png and prey_15.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 9ms/step
Wrote '0.37592942 0.4437086 0.38387135 0.45053643 0.3922074 0.43240896 ' to response file find_15.txt
  fine_tune_images shape = (16, 128, 128, 3) -- fine_tune_labels shape = (16, 3, 2)
  nearest_center: 65% (nearest_center = 10.5 , steps = 16 )
  -0.393 -0.008 -0.107 -0.97 -0.192 -0.096 0.105 0.011 -0.004 -3.861 0.289 1.044 -0.16 -5.436 -0.02 6.255 0.029 0.211 
successes = [1, 1, 0, 0, 0, 1] 
  nearest_center: 68% (nearest_center = 11.0 , steps = 16 )
  -0.391 -0.007 -0.108 -0.97 -0.191 -0.096 0.105 0.009 -0.005 -3.859 0.29 1.045 -0.16 -5.435 -0.02 6.252 0.03 0.212 
successes = [1, 1, 1, 0, 1, 1] 
  nearest_center: 71% (nearest_center = 11.5 , steps = 16 )
  -0.391 -0.008 -0.108 -0.969 -0.191 -0.096 0.104 0.

  -0.393 -0.008 -0.107 -0.97 -0.192 -0.096 0.105 0.011 -0.004 -3.861 0.289 1.044 -0.16 -5.436 -0.02 6.255 0.029 0.211 
successes = [1, 1, 0, 0, 0, 1, 1, 0, 1] 
  nearest_center: 63% (nearest_center = 14.5 , steps = 23 )
  -0.393 -0.009 -0.106 -0.971 -0.19 -0.095 0.103 0.011 -0.006 -3.861 0.291 1.044 -0.161 -5.435 -0.018 6.253 0.027 0.211 
successes = [1, 0, 1, 1, 0, 0, 1] 

step 23

Waiting for camo_23.png and prey_23.txt ... done, elapsed time: 2 seconds.
1/1 [==============================] - 0s 11ms/step
Wrote '0.5348571 0.5764504 0.53444654 0.5718884 0.522988 0.58400726 ' to response file find_23.txt
  fine_tune_images shape = (24, 128, 128, 3) -- fine_tune_labels shape = (24, 3, 2)
  nearest_center: 60% (nearest_center = 14.5 , steps = 24 )
  -0.393 -0.008 -0.107 -0.97 -0.192 -0.096 0.105 0.011 -0.004 -3.861 0.289 1.044 -0.16 -5.436 -0.02 6.255 0.029 0.211 
successes = [1, 1, 0, 0, 0, 1, 1, 0, 1, 1] 
  nearest_center: 60% (nearest_center = 14.5 , steps = 24 )
  -0.391 -0.008 -0.10

  -0.391 -0.009 -0.109 -0.97 -0.191 -0.097 0.102 0.01 -0.006 -3.859 0.291 1.045 -0.162 -5.437 -0.019 6.255 0.029 0.211 
successes = [1, 1, 1, 0, 0, 0, 1] 
  nearest_center: 56% (nearest_center = 17.5 , steps = 31 )
  -0.39 -0.008 -0.107 -0.969 -0.19 -0.095 0.105 0.009 -0.006 -3.861 0.29 1.046 -0.161 -5.435 -0.021 6.254 0.029 0.212 
successes = [1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 

step 31

Waiting for camo_31.png and prey_31.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 9ms/step
Wrote '0.38570964 0.721288 0.37347043 0.72263813 0.36254093 0.7060043 ' to response file find_31.txt
  fine_tune_images shape = (32, 128, 128, 3) -- fine_tune_labels shape = (32, 3, 2)
  nearest_center: 56% (nearest_center = 18.0 , steps = 32 )
  -0.39 -0.009 -0.109 -0.968 -0.19 -0.094 0.103 0.01 -0.004 -3.858 0.289 1.046 -0.162 -5.435 -0.02 6.255 0.029 0.21 
successes = [0, 0, 1, 1, 1, 1, 1] 
  nearest_center: 57% (nearest_center = 18.5 , steps = 32 )
  -0.39 -0.008 -0.107 -0.969 

...

3/3 [==============================] - 0s 129ms/step - loss: 0.0136 - accuracy: 0.7973 - in_disk: 0.4189
  -0.389 -0.011 -0.107 -0.965 -0.186 -0.097 0.107 0.011 -0.006 -3.861 0.29 1.047 -0.161 -5.435 -0.021 6.254 0.025 0.218 
successes = [1, 1, 0, 1, 0, 1, 1, 0, 1, 1] 

step 74

Waiting for camo_74.png and prey_74.txt ... done, elapsed time: 2 seconds.
1/1 [==============================] - 0s 9ms/step
Wrote '0.7241471 0.70893025 0.7611481 0.75057316 0.7296115 0.76250327 ' to response file find_74.txt
  fine_tune_images shape = (75, 128, 128, 3) -- fine_tune_labels shape = (75, 3, 2)
  nearest_center: 56% (nearest_center = 42.0 , steps = 75 )
Running on CPU ONLY!
3/3 [==============================] - 1s 133ms/step - loss: 0.0162 - accuracy: 0.8000 - in_disk: 0.3467
  -0.399 -0.006 -0.109 -0.976 -0.187 -0.088 0.107 0.008 -0.005 -3.858 0.289 1.047 -0.162 -5.435 -0.019 6.249 0.022 0.216 
successes = [0, 0, 0, 0, 1, 0, 1, 0, 1, 0] 
  nearest_center: 56% (nearest_center = 42.0 , steps = 75

3/3 [==============================] - 0s 152ms/step - loss: 0.0218 - accuracy: 0.7875 - in_disk: 0.3875
  -0.391 -0.007 -0.108 -0.976 -0.192 -0.091 0.101 0.012 -0.005 -3.859 0.29 1.046 -0.16 -5.435 -0.02 6.248 0.024 0.217 
successes = [1, 0, 1, 1, 1, 0, 0, 0, 1, 0] 
  nearest_center: 53% (nearest_center = 42.5 , steps = 80 )
Running on CPU ONLY!
3/3 [==============================] - 0s 148ms/step - loss: 0.0130 - accuracy: 0.7500 - in_disk: 0.5000
  -0.386 -0.009 -0.108 -0.977 -0.195 -0.091 0.101 0.007 -0.006 -3.861 0.29 1.048 -0.16 -5.436 -0.019 6.25 0.024 0.22 
successes = [0, 1, 0, 1, 1, 0, 1, 1, 0, 0] starve!
  nearest_center: 53% (nearest_center = 42.5 , steps = 80 )
Running on CPU ONLY!
3/3 [==============================] - 0s 149ms/step - loss: 0.0193 - accuracy: 0.8125 - in_disk: 0.2750
  -0.383 -0.002 -0.109 -0.965 -0.188 -0.094 0.102 0.007 -0.006 -3.859 0.291 1.045 -0.162 -5.437 -0.019 6.249 0.02 0.219 
successes = [1, 0, 0, 1, 0, 0, 1, 0, 1, 1] 

step 80

Waiting for camo

3/3 [==============================] - 1s 165ms/step - loss: 0.0181 - accuracy: 0.7294 - in_disk: 0.3176
  -0.388 -0.006 -0.109 -0.962 -0.194 -0.088 0.1 0.011 -0.004 -3.858 0.289 1.046 -0.162 -5.435 -0.02 6.25 0.023 0.214 
successes = [1, 0, 0, 0, 0, 1, 0, 0, 0, 1] 

step 85

Waiting for camo_85.png and prey_85.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 9ms/step
Wrote '0.57955885 0.6266739 0.5592144 0.6550192 0.55638796 0.65147763 ' to response file find_85.txt
  fine_tune_images shape = (86, 128, 128, 3) -- fine_tune_labels shape = (86, 3, 2)
  nearest_center: 49% (nearest_center = 42.5 , steps = 86 )
Running on CPU ONLY!
3/3 [==============================] - 1s 165ms/step - loss: 0.0142 - accuracy: 0.8023 - in_disk: 0.3256
  -0.383 0.001 -0.106 -0.961 -0.188 -0.092 0.104 0.008 -0.004 -3.86 0.289 1.046 -0.162 -5.436 -0.021 6.251 0.022 0.22 
successes = [1, 1, 0, 1, 1, 1, 1, 0, 0, 1] 
  nearest_center: 49% (nearest_center = 42.5 , steps = 86 )
Run

3/3 [==============================] - 1s 189ms/step - loss: 0.0171 - accuracy: 0.7802 - in_disk: 0.4176
  -0.385 0.003 -0.109 -0.976 -0.193 -0.083 0.115 0.003 -0.006 -3.86 0.29 1.046 -0.161 -5.437 -0.018 6.253 0.025 0.217 
successes = [1, 1, 1, 1, 1, 1, 0, 1, 1, 0] 
  nearest_center: 48% (nearest_center = 44.0 , steps = 91 )
Running on CPU ONLY!
3/3 [==============================] - 1s 183ms/step - loss: 0.0149 - accuracy: 0.8242 - in_disk: 0.4396
  -0.387 -0.003 -0.109 -0.961 -0.195 -0.087 0.098 0.014 -0.004 -3.858 0.289 1.046 -0.162 -5.435 -0.02 6.25 0.023 0.217 
successes = [0, 0, 1, 0, 0, 0, 1, 0, 0, 0] starve!
  nearest_center: 48% (nearest_center = 44.0 , steps = 91 )
Running on CPU ONLY!
3/3 [==============================] - 1s 176ms/step - loss: 0.0154 - accuracy: 0.7692 - in_disk: 0.4725
  -0.381 -0.001 -0.108 -0.962 -0.191 -0.094 0.109 0.007 -0.005 -3.859 0.291 1.044 -0.161 -5.436 -0.02 6.251 0.02 0.22 
successes = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0] starve!

step 91

Waiting f

KeyboardInterrupt: 

In [ ]:
# Normally start from step 0, or if an "other" file exists
# (eg 'camo_123.jpeg') then restart from that point.

# restart_run()

#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#